In [1]:
import numpy as np
from sklearn.metrics.pairwise import pairwise_kernels, check_pairwise_arrays
from sklearn.metrics.pairwise import rbf_kernel as sklearn_kernel
from scipy.stats import norm, uniform
from gekko import GEKKO
from rkhs_utils.utils import rkhs_norm_squared, rkhs_func_exp, rkhs_func
np.set_printoptions(suppress=True)

In [2]:
# Define constraint function

def f_constraint(x, X):
    """
    Constraint function
    
    x: cp.Variable -- decision variable (dim,)
    X: ndarray -- Samples (n_samples, dim)
    """
#     f = cp.exp(0.1 * (x[0, :] + X[:,0])) + x[1, :] + X[:, 1] -10
    
    f = X @ x - 50

    return f

In [3]:
##### Sample/set problem constants
# Define constants and sample scenarios
np.random.seed(1)
dim = 1
n_samples = 20
c = np.random.randint(1, 50, size=dim)
X = uniform.rvs(loc=1, scale=9, size=(n_samples, dim))
M = 100000
epsilon = 0.5
alpha = 0.05
sigma = 0.5

In [4]:
model = GEKKO()
x = model.Array(model.Var, dim,)
w = model.Array(model.Var, n_samples)
t = model.Var(value=0, name='t')
s = model.Var(value=0, lb=0, name='s')

model.Obj(-c @ x)
model.Equation(rkhs_func_exp(w, X, sigma=sigma) + epsilon * s <= t * alpha)
model.Equation(rkhs_norm_squared(w, X, sigma=sigma) <= s**2)
lhs = f_constraint(x, X) + t
rhs = rkhs_func(w, X, X, sigma=sigma)
tmp = lhs - rhs
model.Equations([tmp[i] <= 0 for i in range(tmp.shape[0])])
model.Equations([rhs[i] >= 0 for i in range(rhs.shape[0])])

model.solve(disp=False)

In [5]:
print('Results')
print('x: ', x)
print('w: ', w)
print('t: ', t.value)
print('Objective: ' + str(-model.options.objfcnval))

Results
x:  [[5.0043214281]]
w:  [[-0.031151966489] [0.0086282393169] [6.96319827e-08] [0.029341421668]
 [3.2552825801e-06] [-0.00013293230036] [8.3275317868e-05]
 [-9.0970461943e-06] [-0.063876108557] [-0.00028102460631]
 [-1.5473821695e-06] [-3.8459832482e-05] [-0.00036334522865]
 [-2.9242820454e-06] [3.0834854978e-05] [0.005019210829]
 [0.00015617483421] [0.00026670871739] [0.052354540673] [7.0222945689e-05]]
t:  [1.717057481e-06]
Objective: 190.16421427
